In [ ]:
import os
import numpy as np
from databaseemulator import DataBaseEmulator
from domain_model import StateVariable, Activity
from stats import KDE

In [ ]:
# Load the database with the cut-in scenarios.
filename = os.path.join("data", "5_scenarios", "cut_in_scenarios.json")
cutins = DataBaseEmulator(filename)
for i in [39, 130, 258, 288]:
    
    cutins.delete_item("scenario", i)
nscenarios = len(cutins.collections["scenario"])
print("Number of scenarios: {:d}".format(nscenarios))

# Step 1: Item generation

In [ ]:
# Go through all cut-ins and store the sequence of longitudinal activities.
activities = []
for i in range(nscenarios):
    # Get the acts in chronological order.
    scenario = cutins.get_ordered_item("scenario", i)
    acts = [scenario.acts[i] for i in np.argsort([act[2] for act in scenario.acts])]
    
    # List the longitudinal activities of the target vehicle.
    curr_activities = []
    for act in acts:
        if act[0].name == "target vehicle" and \
                act[1].activity_category.state == StateVariable.LON_TARGET:
            curr_activities.append(act[1].name[0])
    activities.append(curr_activities)

In [ ]:
# Store in a dictionary the possibilities of 'item' sequences.
# 'af' means "accelerating and lane following"
# 'dl' means "decelerating and lane changing"
# 'cf', 'df', 'al', and 'cl' are similarly defined.
def get_transition(activities, main_act, other_acts):
    counter = np.zeros(1+len(other_acts), dtype=np.int)
    for activity in activities:
        try:
            i = activity.index(main_act)
        except ValueError:
            pass
        else:
            if i == len(activity)-1:
                counter[len(other_acts)] += 1
                continue
            for j, other_act in enumerate(other_acts):
                if activity[i+1] == other_act:
                    counter[j] += 1
    counter = counter / np.sum(counter)
    probs = dict()
    probs['end'] = counter[-1]
    for i, other_act in enumerate(other_acts):
        probs[other_act+'l'] = counter[i]
    return probs
transition = dict(af=dict(al=1),
                  cf=dict(cl=1),
                  df=dict(dl=1),
                  al=get_transition(activities, 'a', ['c', 'd']),
                  cl=get_transition(activities, 'c', ['a', 'd']),
                  dl=get_transition(activities, 'd', ['a', 'c']))

In [ ]:
acts, counts = np.unique([activity[0] for activity in activities], return_counts=True)
transition['start'] = dict()
for act, count in zip(acts, counts/np.sum(counts)):
    transition['start'][act+'f'] = count
transition

In [ ]:
def next_item(probs):
    """ It is important that the probabilities in probs add up to 1. """
    rand = np.random.rand()
    for key, value in probs.items():
        if rand <= value:
            return key
        rand -= value

def generate_sequence(transitions, seed=None):
    if seed is not None:
        np.random.seed(0)
    sequence = ['start']
    while not sequence[-1] == 'end':
        sequence.append(next_item(transitions[sequence[-1]]))
    return sequence[1:-1]

def probability_sequence(transitions, sequence):
    sequence = ['start'] + sequence + ['end']
    prob = 1
    for act1, act2 in zip(sequence[:-1], sequence[1:]):
        prob *= transitions[act1][act2]
    return prob

In [ ]:
np.random.seed(0)
n = 10000
seqs = [generate_sequence(transition) for _ in range(n)]

In [ ]:
sequences = [['cf', 'cl'],
             ['af', 'al'],
             ['df', 'dl'],
             ['cf', 'cl', 'al'],
             ['cf', 'cl', 'dl'],
             ['af', 'al', 'cl'],
             ['af', 'al', 'dl'],
             ['df', 'dl', 'al'],
             ['df', 'dl', 'cl']] #,
x=[             ['cf', 'cl', 'al', 'cl'],
             ['cf', 'cl', 'al', 'dl'],
             ['cf', 'cl', 'dl', 'cl'],
             ['cf', 'cl', 'dl', 'al'],
             ['af', 'al', 'cl', 'al'],
             ['af', 'al', 'cl', 'dl'],
             ['af', 'al', 'dl', 'al'],
             ['af', 'al', 'dl', 'cl'],
             ['df', 'dl', 'al', 'cl'],
             ['df', 'dl', 'al', 'dl'],
             ['df', 'dl', 'cl', 'al'],
             ['df', 'dl', 'cl', 'dl']]
probs = [probability_sequence(transition, seq) for seq in sequences]
# .count() is very slow like this, but fine for now...
mc_probs = [seqs.count(seq)/n for seq in sequences]
real_seqs = [[y[0]+'f']+[x+'l' for x in y] for y in activities]
real_probs = [real_seqs.count(seq)/len(real_seqs) for seq in sequences]
for seq, prob, mc_prob, real_prob in zip(sequences, probs, mc_probs, real_probs):
    print("{:13s} {:.4f} {:.4f} {:.4f}".format(' '.join(seq), prob, mc_prob, real_prob))
print()
print("{:13s} {:.4f} {:.4f} {:.4f}".format("Total", np.sum(probs), np.sum(mc_probs), 
                                           np.sum(real_probs)))

# Step 2: Parameter generation

In [ ]:
# Functions for obtaining the important parameters.
def parm_acc_dec(activity: Activity):
    """ Get initial speed, mean acceleration, and speed difference. """
    vstart, vend = activity.get_state(time=[activity.tstart, activity.tend])[0]
    amean = (vend - vstart) / (activity.tend - activity.tstart)
    return vstart, amean, vend-vstart

def parm_cruise(activity: Activity):
    """ Get initial speed, log(duration), and speed difference. """
    vstart, vend = activity.get_state(time=[activity.tstart, activity.tend])[0]
    return vstart, np.log(activity.tend-activity.tstart), vend-vstart

def parm_lc(activity: Activity):
    """ Get initial position, end position, and log(duration). """
    ystart, yend = activity.get_state(time=[activity.tstart, activity.tend])
    return ystart, yend, activity.tend-activity.tstart

In [ ]:
# Get the parameters of the longitudinal activities.
par_acc = []
par_cru = []
par_dec = []
par_lc = []
for i in range(len(cutins.collections["activity"])):
    activity = cutins.get_item("activity", i)
    if activity.name == "acceleration target":
        par_acc.append(parm_acc_dec(activity))
    elif activity.name == "deceleration target":
        par_dec.append(parm_acc_dec(activity))
    elif activity.name == "cruising target":
        par_cru.append(parm_cruise(activity))
    elif activity.name in ["right lane change", "left lane change"]:
        par_lc.append(parm_lc(activity))

In [ ]:
par_initx = np.zeros(nscenarios)
par_egov = np.zeros(nscenarios)
par_tarv = np.zeros(nscenarios)
for i in range(nscenarios):
    scenario = cutins.get_ordered_item("scenario", i)
    for actor in scenario.actors:
        if actor.name == "ego vehicle":
            break
    par_egov[i] = scenario.get_state(actor, StateVariable.SPEED, scenario.time["start"])
    
    for actor in scenario.actors:
        if actor.name == "target vehicle":
            break
    par_tarv[i], par_initx[i] = scenario.get_state(actor, StateVariable.LON_TARGET, 
                                                   scenario.time["start"])

In [ ]:
class Approach1:
    def __init__(self):
        # Create KDEs of initial parameters.
        self.kde_egov = KDE(par_egov)
        self.kde_initx = KDE(par_initx)
        
        # Create KDE of individual lane changes parameters.
        self.kde_lc_ystart = KDE(np.array(par_lc)[:, 0])
        self.kde_lc_yend = KDE(np.array(par_lc)[:, 1])
        self.kde_lc_dt = KDE(np.array(par_lc)[:, 2])
        
        # Create KDE of each of the longitudinal activities.
        self.kde_a = KDE(np.array(par_acc))
        self.kde_c = KDE(np.array(par_cru))
        self.kde_d = KDE(np.array(par_dec))
        
        # Compute the bandwidths
        for kde in (self.kde_egov, self.kde_initx, self.kde_lc_ystart, self.kde_lc_yend,
                    self.kde_lc_dt, self.kde_a, self.kde_c, self.kde_d):
            kde.compute_bandwidth()
    
    def generate(self, items):
        inits = [self.kde_egov.sample()[0][0], self.kde_initx.sample()[0][0],
                 np.random.rand()]
        lc = [self.kde_lc_ystart.sample()[0][0], self.kde_lc_yend.sample()[0][0], 
              self.kde_lc_dt.sample()[0][0]]
        lon = []
        for item in items[1:]:
            if item[0] == "a":
                lon.append(self.kde_a.sample()[0])
            elif item[0] == "c":
                lon.append(self.kde_c.sample()[0])
                lon[-1][1] = np.exp(lon[-1][1])
            else:
                lon.append(self.kde_d.sample()[0])
        return (inits, lc, lon)
A1 = Approach1()

In [ ]:
np.random.seed(4)
sequence = generate_sequence(transition)
parameters = A1.generate(sequence)
while not valid_cutin(sequence, parameters, verbose=True):
    parameters = A1.generate(sequence)
print(sequence)
print(parameters)
valid_cutin(sequence, parameters)

In [ ]:
def valid_cutin(sequence, parameters, verbose=False):
    # Ego speed should be positive.
    if parameters[0][0] < 0:
        if verbose:
            print("Negative ego speed")
        return False
    
    # Start y should be at least 2 m away from ego lane. End should be less than 1.5 m.
    if np.abs(parameters[1][0]) < 2 or np.abs(parameters[1][1]) > 1.5:
        if verbose:
            print("Wrong lane change parameters")
        return False
    
    durations = np.zeros(len(sequence) - 1)
    for i, (item, par) in enumerate(zip(sequence[1:], parameters[2])):
        # Each longitudinal activity should be longer than 0. 
        # For acc/dec, it means than amean is pos/neg.
        if (item[0] == "a" and par[1] <= 0) or (item[0] == "d" and par[1] >= 0):
            if verbose:
                print("Wrong acceleration/deceleration")
            return False
        if item[0] == "c" and par[1] <= 0:
            if verbose:
                print("Wrong cruising", sequence, parameters)
            return False
        
        # With acceleration/deceleration, there should be a speed increase/decrease.
        if (item[0] == "a" and par[2] <= 0) or (item[0] == "d" and par[2] >= 0):
            return False
        
        if item[0] in ["a", "d"]:
            durations[i] = par[2] / par[1]
        else:
            durations[i] = par[1]
        
    # Last lon activity should start before end lane change and end after it.
    total_duration = -durations[0] * parameters[0][2]
    total_duration += np.sum(durations[:-2])
    if total_duration > parameters[1][2]:
        if verbose:
            print("Longitudinal activities too long")
        return False
    if total_duration + durations[-1] < parameters[1][2]:
        if verbose:
            print("Longitudinal activities too short")
        return False
    
    return True
valid_cutin(sequence, parameters)

In [ ]:
plt.hist(np.log(np.array(par_cru)[:, 1]))